In [ ]:
import os
import glob
import json
import geojson
from osgeo import gdal
import rasterio as rio
from rasterio.enums import Resampling
import tqdm
from pathlib import Path

In [ ]:
year = "2023"
dl_folder = os.path.join("./Sentinel-2", year)

channels = ['B01',
            'B02',
            'B03',
            'B04',
            'B05',
            'B06',
            'B07',
            'B08',
            'B8A',
            'B09',
            'B11',
            'B12',
            'CLDPRB']

SAFEs = glob.glob(os.path.join(dl_folder, f"*R051*.SAFE/GRANULE"))

n_safe = len(SAFEs)

for i, SAFE in enumerate(SAFEs):
    print(i+1, "/", n_safe)

    jp2s = []
    for channel in channels:
        jp2s.append(glob.glob(os.path.join(SAFE, f"**/*{channel}*.jp2"), recursive=True)[0])
    jp2s_n10 = [f for f in jp2s if f.find("10m") == -1]
     
    xres, yres = 10, 10
    for jp2 in jp2s_n10:
        with rio.open(jp2) as dataset:
            scale_factor_x = dataset.res[0]/xres
            scale_factor_y = dataset.res[1]/yres
            
            profile = dataset.profile.copy()
            data = dataset.read(
                out_shape=(
                        dataset.count,
                        int(dataset.height * scale_factor_y),
                        int(dataset.width * scale_factor_x)
                ),
                resampling=Resampling.bilinear)
            transform = dataset.transform * dataset.transform.scale(
                (1 / scale_factor_x),
                (1 / scale_factor_y))
            profile.update({"height": data.shape[-2],
                            "width": data.shape[-1],
                            "transform": transform})

        outfn = jp2.replace('20m', '10m')
        outfn = outfn.replace('60m', '10m')
        with rio.open(outfn, "w", **profile) as dataset:
            dataset.write(data)

    bands = []
    for channel in channels:
        bands.append(glob.glob(os.path.join(SAFE, f"**/*{channel}*10m.jp2"), recursive=True)[0])
    
    geo_info = rio.open(bands[1]).meta
    geo_info.update({"driver": "GTiff",
                     "count": len(bands)})

    output = ".".join(SAFE.split(".")[:-1])+".tiff"

    with rio.open(output, 'w', **geo_info) as dest:
        for i, img in enumerate(bands):
            band = rio.open(img)
            dest.write(band.read(1),i+1)
    print(output)

In [ ]:
def json_latlon_correction(json_file):
    json_file = json.loads(json_file)
    for i in range(len(json_file["coordinates"][0])):
        coord_pair = json_file["coordinates"][0][i]
        json_file["coordinates"][0][i] = coord_pair[::-1]
    return json_file

def bbox(poly):
    coord = poly['geometry']['coordinates']
    xmin = min([i[0] for i in coord[0]])
    ymin = min([i[1] for i in coord[0]])
    xmax = max([i[0] for i in coord[0]])
    ymax = max([i[1] for i in coord[0]])
    return [xmin, ymin, xmax, ymax]

extent_file = "../geodata/32vnm.geojson"

with open(extent_file) as f:
    gj = geojson.load(f)
extentSRS = "EPSG:" + gj['crs']['properties']['name'].split(':')[-1]
extent = gj['features'][0]
print(extentSRS)
envelope = bbox(extent)
print(envelope)

# img_folder = dl_folder
all_tiff = glob.glob(f"{dl_folder}/*R051*.tiff", recursive=False)

options = {
    "format": "GTiff",
    "xRes": 10,
    "yRes": 10,
    "outputBoundsSRS": extentSRS,
    "outputBounds": envelope,
    #"cutlineDSName": extent,
    #"cropToCutline": True,
    "dstSRS": extentSRS+"-tap",
    "resampleAlg": "near",
}

for i, img in enumerate(all_tiff):
    print(i+1, "/", len(all_tiff))
    input_raster = gdal.Open(img)
    pre, ext = os.path.splitext(img)
    output_raster = pre + "_aligned"+ ext

    ds_aligned = gdal.Warp(output_raster, input_raster, **options)
    # print(ds_aligned.GetGeoTransform())
    print(output_raster)
    del ds_aligned

In [ ]:
sensor_name = "Sentinel-2"
year = "2020"
scenes = glob.glob(os.path.join(f"/filserver/frrov/imagery/{sensor_name}/{year}/", '*_aligned.tiff'))

archive = f"/filserver/frrov/imagery/{sensor_name}/{year}/"

for scene in tqdm(scenes):
    with rio.open(scene) as in_ds:
        meta = in_ds.meta
        meta.update(count=12)
        image_name = Path(scene).stem
        with rio.open(os.path.join(archive, image_name + "_12b.tiff"), 'w', **meta) as out_ds:
                    out_ds.write(in_ds.read(tuple(range(1,13))))